In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
from mafContrib import Plasticc_metric, plasticc_slicer
from lsst.sims.utils import _healbin
import healpy as hp

In [ ]:
#runName = 'baseline_1exp_pairsmix_10yrs'
runName = 'roll_mod3_sdf0.20mixed_10yrs.db'
#runName = 'very_alt3_rm5illum20_10yrs'
#This next command assumes you have downloaded baseline2018a.db into the tutorials directory in this repo.
#Change this path according to your datbase and where it is saved.
opsdb = db.OpsimDatabase(runName+'.db')
outDir = 'temp'
resultsDb = db.ResultsDb(outDir=outDir)

In [ ]:
bundleList = []
sql=''
slicer1 = plasticc_slicer(model='SNIa-normal')
slicer2 = plasticc_slicer(model='KN', seed=43)


In [ ]:
metric = Plasticc_metric(metricName='SNIa')
# Set the maskval so that we count missing objects as zero.
summary_stats = [metrics.MeanMetric(maskVal=0)]
bundle = metricBundles.MetricBundle(metric, slicer1, sql, runName=runName, summaryMetrics=summary_stats)
bundleList.append(bundle)

metric = Plasticc_metric(metricName='KN')
bundle = metricBundles.MetricBundle(metric, slicer2, sql, runName=runName, summaryMetrics=summary_stats)
bundleList.append(bundle)



In [ ]:
bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
bgroup = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll()

In [ ]:
bgroup.plotAll(closefigs=False)

In [ ]:
for key in bundleDict:
    print(key, bundleDict[key].summaryValues)


In [ ]:
ack = _healbin(slicer1.slicePoints['ra'],slicer1.slicePoints['dec'],
               bundleDict['SNIa_WellFit'].metricValues, nside=8,
               reduceFunc=np.sum)
              

In [ ]:
hp.mollview(ack)

In [ ]:
plt.plot(slicer2.slicePoints['plc'][0]['r']['mjd'], -2.5*np.log10(slicer2.slicePoints['plc'][0]['r']['fluxcal'])+27.5 )
plt.ylim([35,19])